In [4]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load dataset (train.csv must be present)
df = pd.read_csv("train.csv")

# Select required features
columns = [
    "OverallQual",
    "GrLivArea",
    "TotalBsmtSF",
    "GarageCars",
    "YearBuilt",
    "Neighborhood",
    "SalePrice"
]

df = df[columns]

# Handle missing values
df["TotalBsmtSF"].fillna(df["TotalBsmtSF"].median(), inplace=True)
df["GarageCars"].fillna(df["GarageCars"].median(), inplace=True)
df["Neighborhood"].fillna(df["Neighborhood"].mode()[0], inplace=True)

# Encode Neighborhood manually
neighborhood_mapping = {
    name: idx for idx, name in enumerate(df["Neighborhood"].unique())
}

df["Neighborhood"] = df["Neighborhood"].map(neighborhood_mapping)

# Split features and target
X = df.drop("SalePrice", axis=1)
y = df["SalePrice"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluation
y_pred = model.predict(X_test)

print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R²:", r2_score(y_test, y_pred))

# Save model
with open("house_price_model.pkl", "wb") as f:
    pickle.dump((model, scaler, neighborhood_mapping), f)

# Reload test
with open("house_price_model.pkl", "rb") as f:
    loaded_model, loaded_scaler, loaded_mapping = pickle.load(f)

sample = pd.DataFrame([{
    "OverallQual": 7,
    "GrLivArea": 1800,
    "TotalBsmtSF": 900,
    "GarageCars": 2,
    "YearBuilt": 2005,
    "Neighborhood": list(loaded_mapping.values())[0]
}])

sample_scaled = loaded_scaler.transform(sample)
print("Reloaded model prediction:", loaded_model.predict(sample_scaled))


MAE: 25304.423346372318
MSE: 1582978466.4232862
RMSE: 39786.661916065364
R²: 0.7936230580467322
Reloaded model prediction: [225936.00844683]


/tmp/ipython-input-385806019.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["TotalBsmtSF"].fillna(df["TotalBsmtSF"].median(), inplace=True)
/tmp/ipython-input-385806019.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inpl